In [ ]:
import pandas as pd

from tvDatafeed import TvDatafeed, Interval

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
import time

In [2]:
tv = TvDatafeed()

# portfolio of stocks
stocks = ['SPY']

num_stocks = len(stocks)

# Set the dates
start_date = datetime(2020, 1, 1)
today = datetime.now()
days_difference = (today - start_date).days

# Request more bars to account for weekends/holidays
n_bars = min(days_difference + 100, 5000)  # max 5000 as per API limitations

you are using nologin method, data you access may be limited


In [3]:
# TV requires the exchange information
stock_exchanges = {'SPY': 'AMEX'}

# Function to download data with retry
def download_with_retry(symbol, exchange, retries=3, delay=2):
    for attempt in range(retries):
        try:
            print(f"Downloading {symbol} from {exchange} (attempt {attempt+1})")
            data = tv.get_hist(
                symbol=symbol, 
                exchange=exchange, 
                interval=Interval.in_daily,
                n_bars=1000  # around 4 years of data
            )
            if data is not None and not data.empty:
                print(f"✅ {symbol}: {len(data)} rows downloaded")
                return data
        except Exception as e:
            print(f"❌ Error downloading {symbol}: {e}")
        
        if attempt < retries - 1:
            time.sleep(delay)
    
    print(f"❌ Failed to download {symbol} after {retries} attempts")
    return None

In [4]:
# Download data
print("🚀 Starting data download...")
data_dict = {}
failed_downloads = []

for stock, exchange in stock_exchanges.items():
    data = download_with_retry(stock, exchange)
    if data is not None:
        # Take only the Close column and rename it
        data_dict[stock] = data['close']
    else:
        failed_downloads.append(stock)

if failed_downloads:
    print(f"⚠️ Failed to download: {failed_downloads}")

# Combine into a DataFrame
if data_dict:
    df = pd.DataFrame(data_dict)
    print(f"📊 Combined DataFrame shape: {df.shape}")
    print(f"📅 Date range: {df.index.min()} to {df.index.max()}")
    print("\nFirst 5 rows:")
    print(df.head())
else:
    print("❌ No data downloaded successfully")

🚀 Starting data download...
✅ SPY: 1000 rows downloaded
📊 Combined DataFrame shape: (1000, 1)
📅 Date range: 2021-08-31 16:30:00 to 2025-08-25 16:30:00

First 5 rows:
                        SPY
datetime                   
2021-08-31 16:30:00  451.56
2021-09-01 16:30:00  451.80
2021-09-02 16:30:00  453.19
2021-09-03 16:30:00  453.08
2021-09-07 16:30:00  451.46


In [5]:
df.head()
# df.info()

,SPY
datetime,
2021-08-31 16:30:00,451.56
2021-09-01 16:30:00,451.80
2021-09-02 16:30:00,453.19
2021-09-03 16:30:00,453.08
2021-09-07 16:30:00,451.46


In [6]:
# download to csv
df.to_csv("spy_historical_data.csv")